# Feature engineering on time series

## Load data

In [1]:
path = '../../data/FRED/FRED_joined.parquet'

In [2]:
import pandas as pd

df = pd.read_parquet(path)
df = df[['MR', 'CPI']].copy()

df

,MR,CPI
date,,
1948-01-01,NaN,NaN
1948-02-01,NaN,NaN
...,...,...
2025-09-11,6.35,3.419811
2025-09-12,6.35,3.419811


## Linear regression

### Visualize relationship

In [3]:
import plotly.express as px
fig = px.scatter(df, x="MR", y="CPI", trendline='ols', width=800, height=600)

fig

### Regression summary

In [4]:
r = px.get_trendline_results(fig)
r = r.iloc[0,0].summary2()
r

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
===================================================================
Model:              OLS              Adj. R-squared:     0.446     
Dependent Variable: y                AIC:                17898.9720
Date:               2025-10-06 19:58 BIC:                17911.7584
No. Observations:   4417             Log-Likelihood:     -8947.5   
Df Model:           1                F-statistic:        3556.     
Df Residuals:       4415             Prob (F-statistic): 0.00      
R-squared:          0.446            Scale:              3.3668    
----------------------------------------------------------------------
           Coef.     Std.Err.       t       P>|t|     [0.025    0.975]
----------------------------------------------------------------------
const      0.3420      0.0702     4.8745    0.0000    0.2045    0.4796
x1         0.5345      0.0090    59.6296    0.0000    0.5169    0.5520
-------------------------------------------------------------------
Omnibus:             863.104       Durbin-Watson:          1.406   
Prob(Omnibus):       0.000         Jarque-Bera (JB):       1834.460
Skew:                1.137         Prob(JB):               0.000   
Kurtosis:            5.190         Condition No.:          20      
===================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors
is correctly specified.
"""

## Filter data

,CPI,MR
date,,
2007-01-01,3.136734,6.18
2007-01-04,3.139873,6.18
...,...,...
2009-12-24,1.220101,5.05
2009-12-31,1.195432,5.14


In [6]:
fig = px.scatter(df, x="MR", y="CPI", trendline='ols', width=800, height=600)
fig

In [7]:
r = px.get_trendline_results(fig)
r = r.iloc[0,0].summary2()
r

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
=================================================================
Model:              OLS              Adj. R-squared:     0.506   
Dependent Variable: y                AIC:                149.8142
Date:               2025-10-06 19:58 BIC:                156.2871
No. Observations:   188              Log-Likelihood:     -72.907 
Df Model:           1                F-statistic:        192.4   
Df Residuals:       186              Prob (F-statistic): 1.70e-30
R-squared:          0.508            Scale:              0.12853 
-------------------------------------------------------------------
           Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
-------------------------------------------------------------------
const     -0.9512     0.2496   -3.8116   0.0002   -1.4435   -0.4589
x1         0.5932     0.0428   13.8694   0.0000    0.5088    0.6776
-----------------------------------------------------------------
Omnibus:              43.982       Durbin-Watson:          1.816 
Prob(Omnibus):        0.000        Jarque-Bera (JB):       15.612
Skew:                 -0.480       Prob(JB):               0.000 
Kurtosis:             1.966        Condition No.:          57    
=================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the
errors is correctly specified.
"""

## Feature engineering

### Discretize temporal column

,CPI,MR,period
date,,,
2007-01-01,3.136734,6.18,NaN
2007-01-04,3.139873,6.18,NaN
...,...,...,...
2009-12-24,1.220101,5.05,NaN
2009-12-31,1.195432,5.14,NaN


In [9]:
fig = px.scatter(
    data_frame=df.dropna(),
    x="MR", y="CPI", trendline='ols',
    color='period', facet_col='period'
)

fig.update_layout(width=1200, height=400)
fig.update_xaxes(matches=None)
fig.update_yaxes(matches=None)

fig

In [10]:
r = px.get_trendline_results(fig).set_index('period')
r

,px_fit_results
period,
Before,<statsmodels.regression.linear_model.Regressio...
During,<statsmodels.regression.linear_model.Regressio...
After,<statsmodels.regression.linear_model.Regressio...


In [11]:
from modules import utils
df_results = utils.collect_lr_results(r.px_fit_results)

df_results

,Coef,StdErr,t,p,R²,n
Period,,,,,,
Before,-0.334342,0.048049,-6.958392,5.047275e-08,0.587475,36
During,0.486619,0.044029,11.052174,9.981900e-14,0.753316,42
After,0.351124,0.027444,12.794007,6.663134e-16,0.799694,43


### Reduce granularity

,CPI,MR,period
date,,,
2007-01-01,3.138303,6.180,NaN
2007-01-11,3.144581,6.220,NaN
...,...,...,...
2009-12-16,1.232435,4.995,NaN
2009-12-26,1.195432,5.140,NaN


In [14]:
fig = px.scatter(
    data_frame=df.dropna(),
    x="MR", y="CPI", trendline='ols',
    color='period', facet_col='period'
)

fig.update_layout(width=1200, height=400)
fig.update_xaxes(matches=None)
fig.update_yaxes(matches=None)

fig

In [15]:
r = px.get_trendline_results(fig).set_index('period')
r

,px_fit_results
period,
Before,<statsmodels.regression.linear_model.Regressio...
During,<statsmodels.regression.linear_model.Regressio...
After,<statsmodels.regression.linear_model.Regressio...


In [16]:
df_results = utils.collect_lr_results(r.px_fit_results)
df_results

,Coef,StdErr,t,p,R²,n
Period,,,,,,
Before,-0.326453,0.067017,-4.871173,1.060734e-04,0.555330,21
During,0.504086,0.061600,8.183163,4.045715e-08,0.752709,24
After,0.365826,0.036379,10.056017,4.406968e-10,0.808189,26


baseline                                                   \
            Coef    StdErr          t             p        R²   n   
Period                                                              
Before -0.334342  0.048049  -6.958392  5.047275e-08  0.587475  36   
During  0.486619  0.044029  11.052174  9.981900e-14  0.753316  42   
After   0.351124  0.027444  12.794007  6.663134e-16  0.799694  43   

       granularity_reduced                                                   
                      Coef    StdErr          t             p        R²   n  
Period                                                                       
Before           -0.326453  0.067017  -4.871173  1.060734e-04  0.555330  21  
During            0.504086  0.061600   8.183163  4.045715e-08  0.752709  24  
After             0.365826  0.036379  10.056017  4.406968e-10  0.808189  26

### Transform numerical columns

#### Rolling windows

,CPI,MR,period,CPI_roll
date,,,,
2007-01-01,3.138303,6.180,NaN,NaN
2007-01-11,3.144581,6.220,NaN,NaN
...,...,...,...,...
2009-12-16,1.232435,4.995,NaN,1.411987
2009-12-26,1.195432,5.140,NaN,1.378816


#### Create lag features

,CPI,MR,period,CPI_roll,CPI_lag
date,,,,,
2007-01-01,3.138303,6.180,NaN,NaN,2.748785
2007-01-11,3.144581,6.220,NaN,NaN,2.697956
...,...,...,...,...,...
2009-12-16,1.232435,4.995,NaN,1.411987,NaN
2009-12-26,1.195432,5.140,NaN,1.378816,NaN


In [21]:
utils.plot_explanatory_regression_grid(
    df,
    target="MR",
    explanatory_baseline="CPI",
    explanatory_transformed="CPI_lag",
    categorical="period"
)

#### Compare model performance

#### From plotly library

In [22]:
results = {}

xs = ['CPI', 'CPI_roll', 'CPI_lag']

for x in xs:
    f = px.scatter(
        data_frame=df.dropna(), x="MR", y=x,
        trendline='ols', color='period', facet_col='period'
    )

    r = px.get_trendline_results(f).set_index('period')

    results[x] = utils.collect_lr_results(r.px_fit_results)

pd.concat(results).style

#### From statsmodels library

In [23]:
import statsmodels.formula.api as smf
import pandas as pd

df_model = df.dropna()

baseline = smf.ols("MR ~ CPI * C(period)", data=df_model).fit()
with_rolling = smf.ols("MR ~ CPI_roll * C(period)", data=df_model).fit()
with_lags = smf.ols("MR ~ CPI_lag * C(period)", data=df_model).fit()

df_comparison = pd.DataFrame({
    'Model': ['Baseline', 'With Rolling', 'With Lags'],
    'R²': [baseline.rsquared, with_rolling.rsquared, with_lags.rsquared],
    'Adj R²': [baseline.rsquared_adj, with_rolling.rsquared_adj, with_lags.rsquared_adj],
    'AIC': [baseline.aic, with_rolling.aic, with_lags.aic]
})

df_comparison

,Model,R²,Adj R²,AIC
0,Baseline,0.889388,0.880880,-25.157163
1,With Rolling,0.895719,0.887697,-29.341361
2,With Lags,0.832311,0.819412,4.384820
